In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [41]:
df = pd.read_csv("https://raw.githubusercontent.com/tassalor1/Bank-Term-Deposit-Prediction/Virginia's-Branch/Bank-Additional-full.csv")

In [42]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,year,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,2008,...,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,2008,...,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,2008,...,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,2008,...,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,2008,...,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [43]:
# Show rows and columns
df.shape

(41188, 22)

In [44]:
df.corr

<bound method DataFrame.corr of        age          job  marital            education  default housing loan   
0       56    housemaid  married             basic.4y       no      no   no  \
1       57     services  married          high.school  unknown      no   no   
2       37     services  married          high.school       no     yes   no   
3       40       admin.  married             basic.6y       no      no   no   
4       56     services  married          high.school       no      no  yes   
...    ...          ...      ...                  ...      ...     ...  ...   
41183   73      retired  married  professional.course       no     yes   no   
41184   46  blue-collar  married  professional.course       no      no   no   
41185   56      retired  married    university.degree       no     yes   no   
41186   44   technician  married  professional.course       no      no   no   
41187   74      retired  married  professional.course       no     yes   no   

         contact mo

In [45]:
# Check null values
missing = df.isnull().sum()
missing

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
year              0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [46]:
# Check unknown count per column
for col in df.columns:
    unknown_count = df[col].value_counts().get("unknown")
    print(f"{col}: {unknown_count}")

age: None
job: 330
marital: 80
education: 1731
default: 8597
housing: 990
loan: 990
contact: None
month: None
year: None
day_of_week: None
duration: None
campaign: None
pdays: None
previous: None
poutcome: None
emp.var.rate: None
cons.price.idx: None
cons.conf.idx: None
euribor3m: None
nr.employed: None
y: None


In [47]:
# replacing "unknown" with NaN 
df = df.replace({'unknown' : np.nan, 'unknown' : np.nan})

I will remove rows with a minimal number of missing values per column or eliminate rows in columns that are not considered significant.

In [48]:
#Removed NaN values from these significant columns to prevent potential 
#biases in the dataset that could arise from using imputation methods
df = df.dropna(subset=['marital', 'job'])

In [49]:
# WIll fill in NaN values with mode for these 'housing' & 'loan'
df['housing'].fillna(df['housing'].mode()[0], inplace = True)
df['loan'].fillna(df['loan'].mode()[0], inplace = True)

In [65]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
year              0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

# Find values for each column

In [51]:
df['education'].unique

<bound method Series.unique of 0                   basic.4y
1                high.school
2                high.school
3                   basic.6y
4                high.school
                ...         
41183    professional.course
41184    professional.course
41185      university.degree
41186    professional.course
41187    professional.course
Name: education, Length: 40787, dtype: object>

I will use KNN as its a ordinal categorical variable

In [52]:
df['default'].unique

<bound method Series.unique of 0         no
1        NaN
2         no
3         no
4         no
        ... 
41183     no
41184     no
41185     no
41186     no
41187     no
Name: default, Length: 40787, dtype: object>

In [53]:
df['housing'].unique

<bound method Series.unique of 0         no
1         no
2        yes
3         no
4         no
        ... 
41183    yes
41184     no
41185    yes
41186     no
41187    yes
Name: housing, Length: 40787, dtype: object>

In [54]:
df['loan'].unique

<bound method Series.unique of 0         no
1         no
2         no
3         no
4        yes
        ... 
41183     no
41184     no
41185     no
41186     no
41187     no
Name: loan, Length: 40787, dtype: object>

I'm using mode imputation to replace the unknown values in the 'default', 'housing', and 'loan' columns

# Replace 'unknown' values

In [58]:
# Convert the 'education' column to numerical values
encoder = LabelEncoder()
default_encoded = encoder.fit_transform(df['default'].astype(str))

In [62]:
# Convert 'education' to numerical values
df['education'] = df['education'].replace(['illiterate', 'basic.4y', 'basic.6y','basic.9y','high.school','professional.course','university.degree'], [0,1,2,3,4,5,6])
education_encoded = encoder.fit_transform(df['education'].astype(str))

In [63]:
# Apply KNNImputer to fill the missing values in the encoded 'default' column
imputer = KNNImputer(n_neighbors=5)
default_imputed = imputer.fit_transform(default_encoded.reshape(-1, 1))

# Round the imputed values to the nearest integer
default_imputed = np.round(default_imputed).astype(int)

# Convert the imputed numerical values back to their original category names
default_imputed = encoder.inverse_transform(default_imputed.ravel())

# Replace the original 'education' column with the imputed one 
df.loc[:, 'default'] = default_imputed

In [64]:
# Apply KNNImputer to fill the missing values in the encoded 'education' column
imputer = KNNImputer(n_neighbors=5)
education_imputed = imputer.fit_transform(education_encoded.reshape(-1, 1))

# Round the imputed values to the nearest integer
education_imputed = np.round(education_imputed).astype(int)

# Convert the imputed numerical values back to their original category names
education_imputed = encoder.inverse_transform(education_imputed.ravel())

# Replace the original 'education' column with the imputed one
df.loc[:, 'education'] = education_imputed

In [67]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
year              0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

Thats all NaN values sorted

In [68]:
df.describe()

,age,year,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,40787.000000,40787.000000,40787.000000,40787.000000,40787.000000,40787.000000,40787.000000,40787.000000,40787.000000,40787.000000,40787.000000
mean,39.978817,2008.378062,258.329811,2.566112,-0.743325,0.172874,0.080516,93.574804,-40.515770,3.619532,5167.017866
std,10.402157,0.578813,259.269596,2.768103,1.507625,0.494863,1.570133,0.578558,4.626805,1.734416,72.231843
min,17.000000,2008.000000,0.000000,1.000000,-1.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,2008.000000,102.000000,1.000000,-1.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,2008.000000,180.000000,2.000000,-1.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,2009.000000,319.500000,3.000000,-1.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,2010.000000,4918.000000,56.000000,27.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


The 'duration' column displays a significant discrepancy between its maximum value (nearly 5000) and its mean value (258), suggesting the presence of outliers in the dataset. To address this issue, we will employ the Interquartile Range (IQR) method to identify and remove the outliers by determining both the lower and upper bounds.

In [ ]:
Q1 = df['duration'].quantile(0.25)
Q3 = df['duration'].quantile(0.75)

# calculate the interquartile range (IQR)
IQR = Q3 - Q1

# define the lower and upper bounds for outlier detection
lower_bound = Q1 - 3*IQR
upper_bound = Q3 + 3*IQR

# identify the outliers in the column
outliers = df[(df['duration'] < lower_bound) | (df['duration'] > upper_bound)]


# remove the outliers from the DataFrame
df = df[(df['duration'] >= lower_bound) & (df['duration'] <= upper_bound)]

In [70]:
# Save cleaned data to new csv for EDA
df.to_csv('cleaned_data.csv', index=False)